# Re-Analysis of behavioral data and comparison with Kolossa model

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 15 20:32:41 2014

@author: moritz
"""

#get the data as np_array of the form chan*time*epochs
from __future__ import division
#re-test for gammas/pnulls
%matplotlib inline
from helper_functions import *

#import re
import os
import pystan
import pandas as pd
import numpy as np
#this is the model

hierarchic_model = """
data {
int<lower=0> npb;
int<lower=0> cases;
int<lower=0> N[npb,cases];
int<lower=0> y[npb,cases];
real X[cases];
}
parameters {
real<lower=0> mu_gam;
real<lower=0> tau_gam;
real mu_pz;
real<lower=0> tau_pz;
real<lower=0> gam[npb];
real pzero[npb];
}
model {

mu_gam ~ normal(1,1) T[0,];
mu_pz ~ normal(0,10);
gam ~ normal(mu_gam,tau_gam);
pzero ~ normal(mu_pz,tau_pz);


for (i in 1:cases)
{
for (j in 1:npb)
y[j,i] ~ binomial_logit(N[j,i],X[i]*gam[j] + (1-gam[j])*pzero[j]);
}
}
"""

hierarchic_model_kolossa = """
data {
int<lower=0> npb;
int<lower=0> cases;
int<lower=0> N[npb,cases];
int<lower=0> y[npb,cases];
real X[cases];
}
parameters {
real<lower=0> mu_gam;
real<lower=0> tau_gam;
real<lower=0> gam[npb];
}
model {

mu_gam ~ normal(1,1) T[0,];
gam ~ normal(mu_gam,tau_gam);


for (i in 1:cases)
{
for (j in 1:npb)
y[j,i] ~ binomial_logit(N[j,i],X[i]*gam[j]);
}
}
"""

hierarchic_model_kolossa2 = """
data {
int<lower=0> npb;
int<lower=0> cases;
int<lower=0> N[npb,cases];
int<lower=0> y[npb,cases];
real X[cases];
}
parameters {
real<lower=0> mu_gam;
real<lower=0> tau_gam;
real<lower=0> gam[npb];
}
model {

mu_gam ~ normal(1,1) T[0,];
gam ~ normal(mu_gam,tau_gam);


for (i in 1:cases)
{
for (j in 1:npb)
y[j,i] ~ binomial_logit(N[j,i],X[i]*gam[j]);
}
}
"""



k_model = """
data {
int<lower=0> N;
real x[N,4];
int<lower=0,upper=1> y[N];
}
parameters {
real<lower=0> gam;
}
model {

gam ~ normal(1,1) T[0,];

for (n in 1:N)
y[n] ~ bernoulli(inv_logit(gam * x[n,4]+x[n,3]*gam^2+x[n,2]*gam^3+x[n,1]*gam^4));
}
"""


k_model2 = """
data {
int<lower=0> N;
real x[N,4];
int<lower=0,upper=1> y[N];
}
parameters {
real<lower=0> gam;
real pz;
}
model {

gam ~ normal(1,1) T[0,];
pz ~ normal(0,1.45);
for (n in 1:N)
y[n] ~ bernoulli(inv_logit(gam * x[n,4]+x[n,3]*gam^2+x[n,2]*gam^3+x[n,1]*gam^4+((1-gam)*pz)+((1-gam)*pz)^2+((1-gam)*pz)^3+((1-gam)*pz)^4));
}
"""

In [2]:
#list the files, assuming a folder structure of cwd/Data/...
files = os.listdir("Data/newdata/")

#now get all the data for exp80
pattern = "80exp"

exp80_y = pd.DataFrame({0:14*[0]})
exp80_n = pd.DataFrame({0:14*[0]})

for fn in files:
    if pattern in fn:
        tmp = pd.read_table("Data/newdata/"+fn,sep=" ",header=None)
        exp80_y[int(fn[5:7])] = tmp[0]
        exp80_n[int(fn[5:7])] = tmp[1]
       
#now sort them
exp80_n.sort_index(axis=1,inplace=True)
exp80_y.sort_index(axis=1,inplace=True)

#for easier use convert to array/CHANGE THIS LATER

exp80_n = exp80_n.as_matrix() 
exp80_y = exp80_y.as_matrix()

#and also drop the first column

exp80_n = exp80_n[:,1:]
exp80_y = exp80_y[:,1:]

#pop first 2 cases, so no first ball
exp80_n = exp80_n[2:,:]
exp80_y = exp80_y[2:,:]
#03a or 03?
#now you can start re-analysing the data
#there are 16 vps
npb = 16
#and 14 (12 without 1st ball) different cases for the binomial distribution
cases = 12

#also, we need the actual posterior probabilities as logits


prior = 0.2
likelihood = 0.7

x = np.array([(prior*(likelihood**j)*((1-likelihood)**(i-j)))/(((1-prior)*((likelihood**(i-j))*((1-likelihood)**j)))+(prior*(likelihood**j)*((1-likelihood)**(i-j)))) for i in xrange(1,5) for j in xrange(i+1)])
lo_x = np.log(x/(1-x))
#and again drop first ball/2 cases
lo_x = lo_x[2:]

#transpose so rows are vps and columns are cases
exp80_n = exp80_n.T
exp80_y = exp80_y.T

## Model with 3 balls, non-unique posteriors

In [ ]:
#%%
#ONLY FOR 3 BALLS AND MODEL WITH NON-UNIQUE POSTERIORS
#create the data-structure
data_exp80 = {"npb" : npb,"cases":cases,"N":exp80_n,"y":exp80_y,"X":lo_x}
#fit now
fit = pystan.stan(model_code=hierarchic_model_kolossa, data=data_exp80,iter=10000, chains=10)

## Model with only unique posteriors

In [ ]:
#%%
#MODEL WITH ONLY UNIQUE POSTERIORS 
#try to compress X, so same values are added
#first round

lo_x = around(lo_x,3)

#now add n and y with the same X (with cool comprehension)

exp80_n = array( [ sum(exp80_n[:,lo_x==lx],1) for lx in unique(lo_x)]).T
exp80_y = array( [ sum(exp80_y[:,lo_x==lx],1) for lx in unique(lo_x)]).T

#update cases
cases = exp80_n.shape[1]

data_exp80 = {"npb" : npb,"cases":cases,"N":exp80_n,"y":exp80_y,"X":unique(lo_x)}

In [ ]:
#%%
fit = pystan.stan(model_code=hierarchic_model, data=data_exp80,
                 iter=40000, chains=20)
#%%

## Comparison with Kolossa model

In [3]:
likelihood = { 1 : 0.3, 2: 0.7 }
prior = 0.2

path = "/home/mboos/Work/Bayesian Updating/Data/"
path_mat = "/home/mboos/Work/Bayesian Updating/Data EEG/"

files = os.listdir(path)
mat_files = os.listdir(path_mat)

pattern_TS = "80exp"
pattern = "exp80"

blist_dict = dict()
brar_dict = dict()

#%%
#strip VEOH,HEOG electrodes

#100ms BEFORE stimulus presentation


for fn in files:
    if fn.startswith("TS") and pattern_TS in fn:
        bclass = get_bclass_list(fn,prior,likelihood)
        brar_dict[fn[4:6]] = get_bclass(fn,prior,likelihood)[:,0]
        blist_dict[fn[4:6]] = bclass

 
for pb in sorted(blist_dict.keys()):
    for i,e in enumerate(blist_dict[pb]):
        blist_dict[pb][i][0] = e[0] + e[1]
        blist_dict[pb][i].pop(1)
        blist_dict[pb][i] = [0 for t in xrange(4-len(blist_dict[pb][i]))] + blist_dict[pb][i]

In [4]:
y = np.concatenate([brar_dict[pb]-1 for pb in brar_dict.keys()])
X = np.vstack([blist_dict[pb] for pb in brar_dict.keys()])
X = X[y>=0,:]
y = y[y>=0]
y = [int(i) for i in y]

In [158]:
k_data = {'N' : X.shape[0],'x':X,'y':y}
fit = pystan.stan(model_code=k_model, data=k_data,iter=5000, chains=10)

/home/mboos/anaconda/lib/python2.7/multiprocessing/queues.py:390: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)
/home/mboos/anaconda/lib/python2.7/multiprocessing/queues.py:390: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)
/home/mboos/anaconda/lib/python2.7/multiprocessing/queues.py:390: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)
/home/mboos/anaconda/lib/python2.7/multiprocessing/queues.py:390: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must b

In [5]:
k_data = {'N' : X.shape[0],'x':X,'y':y}
fit = pystan.stan(model_code=k_model2, data=k_data,iter=10000, chains=10)

/home/mboos/anaconda/lib/python2.7/multiprocessing/queues.py:390: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)
/home/mboos/anaconda/lib/python2.7/multiprocessing/queues.py:390: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)
/home/mboos/anaconda/lib/python2.7/multiprocessing/queues.py:390: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)
/home/mboos/anaconda/lib/python2.7/multiprocessing/queues.py:390: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must b

In [6]:
fit

Inference for Stan model: anon_model_a57465d4dff3e6b6213a670a858663ea.
10 chains, each with iter=10000; warmup=5000; thin=1; 
post-warmup draws per chain=5000, total post-warmup draws=50000.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
gam    0.88  1.6e-4   0.01   0.86   0.88   0.88   0.89   0.91 5700.0    1.0
pz     3.98  7.2e-3   0.54   3.03   3.59   3.95   4.32   5.16 5635.0    1.0
lp__  -1257    0.01   1.01  -1260  -1258  -1257  -1257  -1256 7192.0    1.0

Samples were drawn using NUTS(diag_e) at Wed Aug  5 18:32:44 2015.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).